### LLM Application with LCEL
This is LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. 

Application is built with :

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing application using LangSmith

- Deploying your application with LangServe

In [1]:
! pip install langchain

In [5]:
! pip install langchain_core

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")


'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'

In [3]:
! pip install langchain_groq

In [ ]:
! pip install streamlit

In [4]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023BA069EEF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023BA06C53C0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [7]:
result

AIMessage(content='Here are a few ways to translate "Hello, How are you?" to French:\n\n* **Formal:** \n    * Bonjour, comment allez-vous ?\n* **Informal:**\n    * Salut, comment vas-tu ? (used with friends or people your own age)\n    * Salut, ça va ? (very informal)\n\n\nLet me know if you\'d like more variations!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 21, 'total_tokens': 106, 'completion_time': 0.154545455, 'prompt_time': 0.002089812, 'queue_time': 0.338329067, 'total_time': 0.156635267}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--932d95ef-0953-45cd-b923-892d6bd7b7aa-0', usage_metadata={'input_tokens': 21, 'output_tokens': 85, 'total_tokens': 106})

In [8]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a few ways to translate "Hello, How are you?" to French:\n\n* **Formal:** \n    * Bonjour, comment allez-vous ?\n* **Informal:**\n    * Salut, comment vas-tu ? (used with friends or people your own age)\n    * Salut, ça va ? (very informal)\n\n\nLet me know if you\'d like more variations!\n'

In [9]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here are a few ways to translate "Hello, How are you?" to French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to say it.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (Used with friends and family)\n* **Coucou, ça va ?** (Very casual, used with close friends)\n\nLet me know if you\'d like to see more variations!\n'

In [10]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [11]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [13]:
result

ChatPromptValue(messages=[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [14]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [16]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"What is the capital of India"})

"Quelle est la capitale de l'Inde ? \n\n\nLet me know if you have any other translations you need! 😊  \n"